## Contextual Compression

In [13]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

AZURE_DEPLOYMENT_NAME= os.environ['AZURE_DEPLOYMENT_NAME']
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.environ['AZURE_OPEN_API_KEY']
os.environ["OPENAI_API_BASE"] = os.environ['AZURE_API_BASE']
os.environ["OPENAI_VERSION"] = "2023-03-15-preview"   
    

In [2]:
import wikipedia
wikipedia.set_lang("ja")
wp = wikipedia.page("こちら葛飾区亀有公園前派出所")
with open('./data/summary.txt', 'w', encoding='utf-8') as f:
   f.write(wp.summary)  
with open('./data/content.txt', 'w', encoding='utf-8') as f:
   f.write(wp.content)

In [9]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

def pretty_print_docs(docs):
    print(f"\n{'-' * 10}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

documents = TextLoader('data/content.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)  
texts = text_splitter.split_documents(documents)
len(texts)


Created a chunk of size 842, which is longer than the specified 300
Created a chunk of size 1694, which is longer than the specified 300
Created a chunk of size 1183, which is longer than the specified 300
Created a chunk of size 426, which is longer than the specified 300
Created a chunk of size 661, which is longer than the specified 300
Created a chunk of size 995, which is longer than the specified 300
Created a chunk of size 1030, which is longer than the specified 300
Created a chunk of size 415, which is longer than the specified 300
Created a chunk of size 1233, which is longer than the specified 300
Created a chunk of size 480, which is longer than the specified 300
Created a chunk of size 898, which is longer than the specified 300
Created a chunk of size 312, which is longer than the specified 300
Created a chunk of size 1829, which is longer than the specified 300
Created a chunk of size 310, which is longer than the specified 300
Created a chunk of size 774, which is longe

32

In [14]:
import openai

# TODO
# embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002",chunk_size=1)
# response = openai.Embedding.create(input=texts, engine="text-embedding-ada-002")
# embeds = [record['embedding'] for record in response['data']]


In [32]:
import openai

from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.callbacks import get_openai_callback
from langchain.chat_models import AzureChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

llm = AzureChatOpenAI(
    client=None,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0,
    request_timeout=180,                    
)


In [17]:
# from langchain.chat_models import ChatOpenAI
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor

# llm = ChatOpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
# compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

# compressed_docs = compression_retriever.get_relevant_documents("信が目指しているものは何ですか？")
# pretty_print_docs(compressed_docs)


In [18]:
# llm_chain.predict(human_input="はいお話ししましょう")